In [ ]:
import requests
import json

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=7TOzdGr7SikuNauga42FVUh84HA0ZS1mvb3AaF3j"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

data=json.loads(response.text)

complexes = data['complexes']

complexe_list=[]
for comp in complexes:
    complexe_list.append({
        "complex_id" : comp["id"],
        "complex_name" : comp["name"]
    })

Venue_list=[]
for comp in complexes:
    for ven in comp.get("venues", []):
        Venue_list.append({
            "id" : ven["id"],
            "name" : ven["name"],
            "city_name" : ven["city_name"],
            "country_name" : ven["country_name"],
            "country_code" : ven["country_code"],
            "timezone" : ven["timezone"],
            "complex_id" : comp["id"]
        })


In [ ]:
import pandas as pd
complexes_df = pd.DataFrame(complexe_list)
complexes_df

In [ ]:
venues_df = pd.DataFrame(Venue_list)
venues_df

In [ ]:
complexes_d= complexes_df.drop_duplicates()
print(complexes_d)

In [ ]:
complexes_d.to_csv("complexes.csv")

In [ ]:
venues_d= venues_df.drop_duplicates()
print(venues_d)

In [ ]:
venues_d.to_csv("venues.csv")

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="tennis_data"   
)

mycursor = mydb.cursor(buffered=True)

In [ ]:
mycursor.execute("""
CREATE TABLE complexes (
    complex_id VARCHAR(50) PRIMARY KEY,
    complex_name VARCHAR(100) NOT NULL
)
""")

In [ ]:
mycursor.execute(""" 
CREATE TABLE venues (
    venue_id VARCHAR(50) PRIMARY KEY,
    venue_name VARCHAR(100) NOT NULL,
    city_name VARCHAR(100) NOT NULL,
    country_name VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    timezone VARCHAR(100) NOT NULL,
    complex_id VARCHAR(50), FOREIGN KEY(complex_id)
    REFERENCES complexes(complex_id) 
)
""")

In [ ]:
from tabulate import tabulate

In [ ]:
complex_query = "INSERT INTO complexes(complex_id,complex_name) VALUES (%s, %s)"

complex_data = complexes_d[['complex_id','complex_name']].values.tolist()

mycursor.executemany(complex_query, complex_data)
mydb.commit()


In [ ]:
mycursor.execute("SELECT COUNT(*) FROM complexes")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.execute("SELECT * FROM complexes")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
venues_d.columns

In [ ]:
venue_query = "INSERT INTO venues(venue_id,venue_name,city_name,country_name,country_code,timezone,complex_id) VALUES (%s, %s, %s, %s, %s, %s, %s)"

venue_data = venues_d[['id','name','city_name','country_name','country_code','timezone','complex_id']].values.tolist()

mycursor.executemany(venue_query, venue_data)
mydb.commit()

In [ ]:
mycursor.execute("SELECT COUNT(*) FROM venues")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
mycursor.execute("SELECT * FROM venues")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# List all venues along with their associated complex name

mycursor.execute("""
SELECT venues.venue_id,venues.venue_name,complexes.complex_name
FROM venues 
JOIN complexes ON complexes.complex_id = venues.complex_id""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
# Count the number of venues in each complex

mycursor.execute("""
SELECT complexes.complex_name,COUNT(*) AS No_of_venues 
FROM venues 
JOIN complexes ON complexes.complex_id = venues.complex_id
GROUP BY complexes.complex_name
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# Get details of venues in a specific country (e.g., Chile)

mycursor.execute("SELECT venue_id,venue_name,city_name,country_name,country_code,timezone FROM venues WHERE country_name = 'Chile'")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# Identify all venues and their timezones

mycursor.execute("SELECT venue_id,venue_name,timezone FROM venues ORDER BY timezone")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# Find complexes that have more than one venue

mycursor.execute("""
SELECT complexes.complex_id,complexes.complex_name,COUNT(*) AS No_of_venues 
FROM venues 
JOIN complexes ON complexes.complex_id = venues.complex_id
GROUP BY complexes.complex_id,complexes.complex_name
HAVING COUNT(*) > 1
""")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# List venues grouped by country

mycursor.execute("SELECT venue_id,venue_name,country_name FROM venues ORDER BY country_name,venue_name")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

In [ ]:
# Find all venues for a specific complex (e.g., Nacional)

mycursor.execute("""
SELECT v.venue_id,v.venue_name,c.complex_name 
FROM venues v
JOIN complexes c ON c.complex_id = v.complex_id
WHERE c.complex_name= 'Am Rothenbaum' """)
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))